In [1]:
from rds_access import execute_sql_query, DatabaseConnection
import os
import pandas as pd

In [2]:
!ls

ad_hoc_query.ipynb     input_data	     pc_platform.py  requirements.txt
ad_hoc_query.py        log_db_benchmark.txt  __pycache__     temp_output
benchmark.py	       msv7.py		     queries.sql
benchmark_results.csv  notebooks	     rds_access.py


In [3]:
conn_params = {                                                   
        "host": os.environ.get("POSTGRE_HOST", ''),                                
        "port": 5432,                                                              
        # "database": "qqmusicv2",                                                 
        # "database": "kugou",                                                     
        # "database": "netease_max",                                               
        "user": os.environ.get("POSTGRE_USER", ''),                                
        "password": os.environ.get("POSTGRE_PWD", '')                              
    }        

In [4]:
pd.set_option('display.max_rows', 5)                                            
pd.set_option('display.max_columns', None) 

In [5]:
conn_params['database'] = "kugou"
# conn_paras['database'] = "netease_max"
# conn_paras['database'] = "qqmusicv2"


In [6]:
sql_query = """SELECT ks.*, ka.album_name
FROM kugou_songs ks
LEFT JOIN kugou_albums ka ON ks.album_id = ka.album_id
WHERE work_name ilike '%lonely%'
"""

In [7]:
with DatabaseConnection(conn_params) as conn:
    df = execute_sql_query(sql_query, conn)
    print(df.shape)

(33286, 14)


In [8]:
df.head()

,audio_id,singer_id,language,ori_author_name,work_name,cid,scrape_date,combine_count,album_id,publish_company,raw_json_1,singer_ids,singer_names,album_name
0,21471941,None,英语,Nat King Cole,Dear Lonely Hearts,254428776,2024-03-16 15:41:40.024000+00:00,0,68535700,Black Barn Music,None,[31289],[Nat King Cole],Somewhere Along the Way
1,21473475,None,英语,Jay McShann & His Orchestra,Lonely Boy Blues,14581258,2024-03-16 15:41:41.045000+00:00,0,12946594,,None,[513784],[Jay McShann & His Orchestra],Hootie's KC Blues
2,21477334,None,英语,Eddie Cochran,Lonely,178485198,2024-03-16 15:41:44.362000+00:00,0,50121536,Golden Ear Music,None,[113305],[Eddie Cochran],Three Steps to Heaven
3,21477449,None,英语,Dion & The Belmonts,Lonely Teenager,131965709,2024-03-16 15:41:44.425000+00:00,0,2253417,Vonotone Records,None,[283830],[Dion & The Belmonts],The Wanderer - Best Of Dion And The Belmonts
4,21480565,None,英语,"Joe ""Fingers"" Webster",Kiss Me Goodnight Sergeant Major / Keep The Ho...,12477738,2024-03-16 15:41:47.376000+00:00,0,46047849,INgrooves,None,[394172],"[Joe ""Fingers"" Webster]",Piano Party Funtime - 69 Sing-Along Favourites


In [9]:
df.to_csv("result100.csv")

In [10]:
df.columns

Index(['audio_id', 'singer_id', 'language', 'ori_author_name', 'work_name',
       'cid', 'scrape_date', 'combine_count', 'album_id', 'publish_company',
       'raw_json_1', 'singer_ids', 'singer_names', 'album_name'],
      dtype='object')

In [11]:
# function to filter the corresponding lines
def row_filter(row):
    return row['ori_author_name'].lower().strip() == 'nana' and 'lonely' in row['work_name'].lower()
    

In [16]:
df_filtered = df.loc[df.apply(row_filter, axis=1)]

In [17]:
df_filtered.to_csv('result_filtered.csv')